## Attention Layer 

In [ ]:
pip install keras==2.1.5

     |████████████████████████████████| 334 kB 4.2 MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


    """Attention layer implementation based in the work of Yang et al. "Hierarchical
    Attention Networks for Document Classification". This implementation also allows
    changing the common tanh activation function used on the attention layer, as Chen
    et al. "A Thorough Examination of the CNN/Daily Mail Reading Comprehension Task"
    point that removing this component can be beneficial to the model. Supports
    masking.
    
    The mathematical formulation of the model is as follows:
    ```
    u = f(W * h + b),
    a_i = softmax(u_i^T * u_s),
    v_i = \sigma_i a_i * h_i.
    ```
    
    Where h are the input tensors with shape (batch, n_timesteps, hidden_size), for
    instance, all hidden vectors produced by a recurrent layer, such as a LSTM and the
    output has shape `(batch, hidden_size)`. This layer also works with inputs with more
    than 3 dimensions as well, such as sentences in a document, where each input has
    size (batch, n_docs, n_sentences, embedding_size), outputing 
    (batch, n_docs, embedding_size)`.
    
    
    # Arguments
        activation: The activation function f used by the layer (see
            [activations](../activations.md)). By default tanh is used, another common
            option is "linear".
        use_bias: Boolean, whether the layer uses a bias vector.
        initializer: Initializer for the `kernel` and `context` matrices
            (see [initializers](../initializers.md)).
        return_attention: If True, instead of returning the sequence descriptor, this
            layer will return the computed attention coefficients for each of the
            sequence timesteps. See Output section for details.
        W_regularizer: Regularizer function applied to the `kernel` weights matrix
            (see [regularizer](../regularizers.md)).
        u_regularizer: Regularizer function applied to the `context` weights matrix
            (see [regularizer](../regularizers.md)).
        b_regularizer: Regularizer function applied to the bias vector
            (see [regularizer](../regularizers.md)).
        W_constraint: Constraint function applied to the `kernel` weights matrix
            (see [constraints](../constraints.md)).
        u_constraint: Constraint function applied to the `contextl` weights matrix
            (see [constraints](../constraints.md)).
        b_constraint: Constraint function applied to the bias vector
            (see [constraints](../constraints.md)).

    # Input shape
        nD tensor with shape: `(batch_size, ..., timesteps, input_dim)`.
        The most common situation would be a 3D input with shape
        `(batch_size, timesteps, input_dim)`.

    # Outuput shape
        The sequence descriptor with shape `(batch_size, ..., timestamps)`. If
        `return_attention` is True, this layer will return the `alpha_i` weights
        for each timestep, and consequently its output shape will be different, namely:
        `(batch_size, ..., timesteps)`.
    """

In [1]:
from keras import backend as K
from keras import initializers
from keras import regularizers
from keras import constraints
from keras.layers import Layer
from keras import initializers
from keras import activations
from sklearn.utils import class_weight

class AttentionLayer(Layer):
    def __init__(self,activation='tanh',initializer='glorot_uniform',return_attention=False,
                 W_regularizer=None,u_regularizer=None,b_regularizer=None,W_constraint=None,u_constraint=None,b_constraint=None,bias=True,**kwargs):
        self.activation = activations.get(activation)
        self.initializer = initializers.get(initializer)
        self.W_regularizer = regularizers.get(W_regularizer)
        self.u_regularizer = regularizers.get(u_regularizer)
        self.b_regularizer = regularizers.get(b_regularizer)
        self.W_constraint = constraints.get(W_constraint)
        self.u_constraint = constraints.get(u_constraint)
        self.b_constraint = constraints.get(b_constraint)
        self.bias = bias
        self.supports_masking = True
        self.return_attention = return_attention
        super().__init__(**kwargs)

    def build(self, input_shape):
        # input_shape: (batch, time, amount_features), the attention size matches the feature dimension
        amount_features, attention_size = input_shape[-1],input_shape[-1]

        self.W = self.add_weight(shape=(amount_features, attention_size),
                                 initializer=self.initializer,regularizer=self.W_regularizer,constraint=self.W_constraint,name='attention_W')
        self.b = None
        if self.bias:
            self.b = self.add_weight(shape=(attention_size,),initializer='zero',regularizer=self.b_regularizer,constraint=self.b_constraint,name='attention_b')

        self.context = self.add_weight(shape=(attention_size,),initializer=self.initializer,regularizer=self.u_regularizer,constraint=self.u_constraint,name='attention_us')

        super().build(input_shape)

    def call(self, x, mask=None):        
        # U = tanh(H*W + b) (eq. 8)        
        ui = K.dot(x, self.W)              # (b, t, a)
        if self.b is not None:
            ui += self.b
        ui = self.activation(ui)           # (b, t, a)

        # Z = U * us (eq. 9: softmax function: exp(ui^tus)/sum(exp(ui^t,us)),us: sentence level context vector, use it to measure sentence importance)
        us = K.expand_dims(self.context)   # (a, 1)
        ui_us = K.dot(ui, us)              # (b, t, a) * (a, 1) = (b, t, 1)
        ui_us = K.squeeze(ui_us, axis=-1)  # (b, t, 1) -> (b, t)
        
        # alpha = softmax(Z) (eq. 9)
        alpha = self._masked_softmax(ui_us, mask) # (b, t), sum(exp(ui^t,us))
        alpha = K.expand_dims(alpha, axis=-1)     # (b, t, 1)
        
        if self.return_attention:
            return alpha
        else:
            # v = alpha_i * x_i (eq. 10)
            return K.sum(x * alpha, axis=1)
    
    def _masked_softmax(self, logits, mask): #https://www.programcreek.com/python/example/93679/keras.backend.cast
        b = K.max(logits, axis=-1, keepdims=True)
        logits = logits - b
        exped = K.exp(logits)

        # ignoring masked inputs
        if mask is not None:
            mask = K.cast(mask, K.floatx()) ## masked timesteps have zero weight
            exped *= mask
        partition = K.sum(exped, axis=-1, keepdims=True)

        # if all timesteps are masked, the partition will be zero. To avoid this issue we use the following trick:
        partition = K.maximum(partition, K.epsilon())
        return exped / partition

    def compute_output_shape(self, input_shape):
        """The attention mechanism computes a weighted average between all hidden vectors generated by the previous sequential layer, hence the input is expected to be
        `(batch_size, seq_len, amount_features)` if `return_attention` is `False`, otherwise the output should be (batch_size, seq_len)."""
        if self.return_attention:
            return input_shape[:-1]
        else:
            return input_shape[:-2] + input_shape[-1:]

    def compute_mask(self, x, input_mask=None):
        """This layer produces a single attended vector from a list of hidden vectors, hence it can't be masked as this means masking a single vector."""
        return None

    def get_config(self):
        config = {'activation': self.activation,'initializer': self.initializer,'return_attention': self.return_attention,
            'W_regularizer': initializers.serialize(self.W_regularizer),'u_regularizer': initializers.serialize(self.u_regularizer),'b_regularizer': initializers.serialize(self.b_regularizer),
            'W_constraint': constraints.serialize(self.W_constraint),'u_constraint': constraints.serialize(self.u_constraint),'b_constraint': constraints.serialize(self.b_constraint),
            'bias': self.bias}

        base_config = super().get_config()
        return dict(list(base_config.items()) + list(config.items()))

In [ ]:
pip install nltk

     |████████████████████████████████| 1.4 MB 4.1 MB/s eta 0:00:01
     |████████████████████████████████| 720 kB 16.9 MB/s eta 0:00:01
     |████████████████████████████████| 75 kB 5.4 MB/s  eta 0:00:01
  Created wheel for nltk: filename=nltk-3.5-py3-none-any.whl size=1434675 sha256=78a64688a46d1f182edae14cc9d36e6dfaf048c9d3823bd659fb0920987e9172
  Stored in directory: /home/jovyan/.cache/pip/wheels/45/6c/46/a1865e7ba706b3817f5d1b2ff7ce8996aabdd0d03d47ba0266
Successfully built nltk
Note: you may need to restart the kernel to use updated packages.


In [2]:
import numpy as np
import matplotlib.pyplot as plt
from keras.datasets import imdb
from keras.preprocessing import sequence
from IPython.display import display, Markdown
from keras import models
from keras import optimizers as opt
from keras.layers import Input
from keras.layers import Embedding
from keras.layers import LSTM
from keras.layers import Dense
import numpy as np
import tensorflow as tf
from tensorflow import keras
import os
import nltk
import string
nltk.download('punkt')
%matplotlib inline

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


## Load Data

### Import data

In [100]:
from google.colab import drive
import pandas as pd 
import string
drive.mount("/content/drive")

#rate the transcript data
train = pd.read_csv("/content/drive/MyDrive/Fordham_research/hate_speech/df_train.csv") #['Tokenized_tweets']
val = pd.read_csv("/content/drive/MyDrive/Fordham_research/hate_speech/df_val.csv") #['Tokenized_tweets']
test = pd.read_csv("/content/drive/MyDrive/Fordham_research/hate_speech/df_test.csv") #['Tokenized_tweets']

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [77]:
data = pd.concat([train,val,test])
data['class'] = data['class'].replace({'Non_Hate':0,'Racist':1,'Offensive':2,'Sexist':3,'religion_offensive':4},inplace=False)

In [78]:
data.head()

,Unnamed: 0,class,tweet,Tokenized_tweets
0,17837,1,@RoseGold_C Okurrr like nigga go daaataway lma...,Okurrr like nigga go daaataway lmao https
1,3506,1,@KarlousM NIGGA SAID JESUS NEPHEW @mynamekayo ...,KarlousM NIGGA SAID JESUS NEPHEW mynamekayo https
2,17507,0,Thinks that Diversity are SOOO much better tha...,Thinks Diversity SOOO much better flawless I t...
3,1874,1,@Pileofblackslag @prince__hubris This faggot d...,Pileofblackslag This faggot doesnt like tummie...
4,447,4,Not sure I want religious symbols on my servos...,Not sure I want religious symbols servos Not I...


In [82]:

#data = data.loc[(data['class']=='Non_Hate') | (data['class']=='Racist')]
#data.loc[data['class']=='Non_Hate','class']=0
#data.loc[data['class']=='Racist','class']=1
# data.loc[data['class']=='Offensive','class']=2
#data.loc[data['class']=='Sexist','class']=1
# data.loc[data['class']=='religion_offensive','class']=4

#Stanford_dataset.loc[Stanford_dataset['class'] == 0,'class'] = 1
samples = list(data['tweet'].values)
labels = list(data['class'].values)
class_names = ['Non_Hate','Racist','Offensive','Sexist','religion_offensive']


### preprocessing

In [83]:
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
vectorizer = TextVectorization(max_tokens=20000, output_sequence_length=200)
text_ds = tf.data.Dataset.from_tensor_slices(samples).batch(128)
vectorizer.adapt(text_ds)
voc = vectorizer.get_vocabulary()
word_index = dict(zip(voc, range(len(voc))))

In [85]:
def tokenize(text):
  transformed_text = []
  stem = nltk.stem.SnowballStemmer('english')
  text = text.lower()
  for token in nltk.word_tokenize(text):
    # if punctuation, pass
    if token in string.punctuation: continue
    if 'http' in token: continue
    if '.com' in token: continue
    if '@' in token: continue
    if 'rt' == token: continue
    # if unknown words, return UNK
    if token not in word_index.keys():transformed_text.append(word_index['[UNK]'])
    # if known words
    if token in word_index.keys():transformed_text.append(word_index[token])
  return transformed_text

In [86]:
# transfer text into word indexes
samples_index = [tokenize(a) for a in samples]


In [87]:
samples_backup = samples
test = np.array(samples)

In [19]:
import tensorflow as tf

In [88]:
#from tf.keras.preprocessing.text import Tokenizer
# Tokenize text data
tokenizer = tf.keras.preprocessing.text.Tokenizer()
tokenizer.fit_on_texts(samples)
samples = tokenizer.texts_to_sequences(samples)

#tokenizer.fit_on_texts(x_test)
#x_test = tokenizer.texts_to_sequences(x_test)


In [90]:
# Shuffle the data
from sklearn.utils import class_weight

seed = 1337
rng = np.random.RandomState(seed)
rng.shuffle(samples)
rng = np.random.RandomState(seed)
rng.shuffle(labels)

# Extract a training & validation split
validation_split = 0.2

num_validation_samples = int(validation_split * len(samples))
train_samples = samples[:-num_validation_samples]
val_samples = samples[-num_validation_samples:]
train_labels = labels[:-num_validation_samples]
val_labels = labels[-num_validation_samples:]

train_samples = np.array(train_samples)
val_samples = np.array(val_samples)
train_labels = np.array(train_labels)
val_labels = np.array(val_labels)
#class_weights = class_weight.compute_class_weight('balanced',np.unique(train_labels),train_labels)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:20: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.


In [91]:
# average length for a given tweet, get this for setting a proper max_length for each tweet
sum([len(item) for item in train_samples])/len(train_samples)

15.259658393492417

In [92]:
# pad text to sequence
VOCAB_SIZE = 200000
MAX_LEN = 128
HIDDEN_SIZE = 100
DROPOUT = 0.5

from keras.utils import pad_sequences
x_train = pad_sequences(train_samples, MAX_LEN, padding='post', truncating='post') # 'post' is adding padding at the end
x_val = pad_sequences(val_samples, MAX_LEN, padding='post', truncating='post') 

## Build Model

In [93]:
def build_model():
    sentence_in = Input((None, ),name='sentence-in') #(shape d:input will be batches of d-dimensional vectors,batch_size,name)
    #(input_dim: Size of the vocabulary, output_dim: Dimension of the dense embedding, 
    #mask_zero: whether the input value 0 is a special "padding" value that should be masked out, index 0 cannot be used in the vocabulary (input_dim should equal size of vocabulary + 1))
    embedded = Embedding(VOCAB_SIZE, HIDDEN_SIZE, mask_zero=True, name='embedding')(sentence_in)
    #(Positive integer, dimensionality of the output space, Whether to return the last output in the output sequence, Fraction of the units to drop for the linear transformation of the inputs)
    vectors  = LSTM(HIDDEN_SIZE*2,return_sequences=True, dropout=DROPOUT,recurrent_dropout=DROPOUT,name='ff-lstm')(embedded)
    vectors2  = LSTM(HIDDEN_SIZE,return_sequences=True, dropout=DROPOUT,recurrent_dropout=DROPOUT,name='ff2-lstm')(vectors) 
    vectors3  = LSTM(HIDDEN_SIZE,return_sequences=True, dropout=DROPOUT,recurrent_dropout=DROPOUT,name='ff3-lstm')(vectors2)
    sentence = AttentionLayer(name='attention')(vectors3) 
    output   = Dense(len(class_names),activation='sigmoid',name='output')(sentence)
    model = models.Model(inputs=[sentence_in], outputs=[output])
    return model

In [94]:
train_labels

array([1, 3, 0, ..., 4, 1, 3])

In [95]:
# (b, t, d)
model = build_model()
model.summary()
model.compile(loss="sparse_categorical_crossentropy", optimizer="rmsprop", metrics=["acc"])
model.fit(x_train, train_labels, batch_size=32, epochs=10, validation_split=0.2)

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 sentence-in (InputLayer)    [(None, None)]            0         
                                                                 
 embedding (Embedding)       (None, None, 100)         20000000  
                                                                 
 ff-lstm (LSTM)              (None, None, 200)         240800    
                                                                 
 ff2-lstm (LSTM)             (None, None, 100)         120400    
                                                                 
 ff3-lstm (LSTM)             (None, None, 100)         80400     
                                                                 
 attention (AttentionLayer)  (None, 100)               10200     
                                                                 
 output (Dense)              (None, 5)                 505 

### Evaluation

In [96]:
from sklearn.metrics import accuracy_score, confusion_matrix,f1_score,recall_score, classification_report

In [122]:
test['class'] = test['class'].replace({'Non_Hate':0,'Racist':1,'Offensive':2,'Sexist':3,'religion_offensive':4},inplace=False)
test_text = test['Tokenized_tweets']
test_labels = test['class']
tokenizer = tf.keras.preprocessing.text.Tokenizer()
tokenizer.fit_on_texts(test_text)
test_token = tokenizer.texts_to_sequences(test_text)

test_samples = pad_sequences(x_val, MAX_LEN, padding='post', truncating='post')

In [127]:
predict = model.predict(test_samples)

376/376 [==============================] - 79s 209ms/step


In [124]:
predict = predict.round().astype(int)

In [118]:
pred = []
for i in predict:
  if (i == [1,0,0,0,0]).all():
    pred.append(0)
  elif (i == [0,1,0,0,0]).all():
    pred.append(1)
  elif (i == [0,0,1,0,0]).all():
    pred.append(2)
  elif (i == [0,0,0,1,0]).all():
    pred.append(3)
  else:
    pred.append(4)

In [ ]:
class_names = ['Non_Hate','Racist','Offensive','Sexist','religion_offensive']

print(classification_report(test_labels, pred))

## Get Attention Weights

In [131]:
def build_viz_model(trained_model):
    """Architecture: input -> embedding -> lstm -> attention -> sigmoid"""
    sentence_in = Input((None, ),name='sentence-in')
    embedded = Embedding(VOCAB_SIZE,HIDDEN_SIZE,mask_zero=True,weights=trained_model.layers[1].get_weights(),name='embedding')(sentence_in)
    vectors  = LSTM(HIDDEN_SIZE,return_sequences=True,dropout=DROPOUT,recurrent_dropout=DROPOUT,weights=trained_model.layers[2].get_weights(),name='ff-lstm')(embedded)
    alphas  = AttentionLayer(weights=trained_model.layers[3].get_weights(),return_attention=True,name='attention')(vectors)
    model = models.Model(inputs=[sentence_in], outputs=[alphas])
    return model

In [ ]:
viz_model = build_viz_model(model)

In [ ]:
def plot_weights(df):
  word = df['word']
  words = [i for i in range(len(word))]
  attention_weight = df['attention_weight']
  plt.plot(words, attention_weight)
  plt.xticks(words, word)
  plt.legend(labels = ['attention weight'])
  plt.show()

In [ ]:
def get_attention(text):
  encoded_txt = np.array([tokenize(text)])
  weights = viz_model(encoded_txt)[0]
  weights = [float(w) for w in weights]
  words = [voc[w] for w in tokenize(text)]
  df = pd.DataFrame([words,weights]).T
  df.columns = ['word','attention_weight']
  return df

In [ ]:
viz_model.save('/content/drive/MyDrive/Project/Tweet Datasets/Model/racist_viz_model')
model.save('/content/drive/MyDrive/Project/Tweet Datasets/Model/racist_trained_model')

vaderSentiment

In [ ]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
 
# function to print sentiments
# of the sentence.
def sentiment_scores(sentence):
 
    # Create a SentimentIntensityAnalyzer object.
    sid_obj = SentimentIntensityAnalyzer()
 
    # polarity_scores method of SentimentIntensityAnalyzer
    # object gives a sentiment dictionary.
    # which contains pos, neg, neu, and compound scores.
    sentiment_dict = sid_obj.polarity_scores(sentence)
     
    print("Overall sentiment dictionary is : ", sentiment_dict)
    print("sentence was rated as ", sentiment_dict['neg']*100, "% Negative")
    print("sentence was rated as ", sentiment_dict['neu']*100, "% Neutral")
    print("sentence was rated as ", sentiment_dict['pos']*100, "% Positive")
 
    print("Sentence Overall Rated As", end = " ")
 